# [User story for my feature](https://github.com/gabrielac07/sprint4_frontend/issues/2)


## Input & Output

### Frontend: Updating the Profile
These input fields allow the user to edit their profile by providing a new username, name, and password. This demonstrates program functionality by enabling users to provide input that directly impacts the program's behavior and updates their profile data.


In [ ]:
<div>
  <label for="newUid">Enter New UID:</label>
  <input type="text" id="newUid" placeholder="New UID">
</div>
<div>
  <label for="newName">Enter New Name:</label>
  <input type="text" id="newName" placeholder="New Name">
</div>
<div>
  <label for="newPassword">Enter New Password:</label>
  <input type="text" id="newPassword" placeholder="New Password">
</div>


### Backend: Adding a Book to Wishlist
This route handles input from a JSON payload (user_id and book_id) and provides meaningful feedback (e.g., success or error messages). It demonstrates input and output by receiving data from the frontend and returning a response that reflects the result of the operation.

In [ ]:
@wishlist_api.route('/', methods=['POST'])
def add_book_to_wishlist():
    if request.is_json:
        data = request.get_json()
        user_id = data.get('user_id')
        book_id = data.get('book_id')

        if not user_id or not book_id:
            return jsonify({"error": "Missing user_id or book_id"}), 400
        ...


## Data Abstraction

### Frontend: Storing Books from a database
The variable predefinedBooks stores a list of books fetched from the backend API. This demonstrates data abstraction by using a variable to represent structured data (book objects) that is dynamically fetched and manipulated.



In [ ]:
let predefinedBooks = [];
async function fetchPredefinedBooks() {
  const URL = `${pythonURI}/api/books`;
  try {
    const response = await fetch(URL, fetchOptions);
    if (!response.ok) throw new Error(`Failed to fetch books: ${response.status}`);
    return await response.json();
  } catch (error) {
    console.error('Error fetching predefined books:', error.message);
    return [];
  }
}


### Backend: Wishlist Table
The wishlist table abstracts data by organizing user wishlist information (user_id and book_id) into a structured format. This demonstrates data abstraction as the program uses the database to store and retrieve relevant data for user wishlists.

In [ ]:
create_wishlist_table = '''
CREATE TABLE IF NOT EXISTS wishlist (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER NOT NULL,
    book_id INTEGER NOT NULL,
    FOREIGN KEY (book_id) REFERENCES books (id)
);
'''

## Algorithm Implementation

### Frontend: Adding a Book to Wishlist
This algorithm takes input from the dropdown, validates the selection, sends a POST request to the backend, and refreshes the wishlist upon success. This demonstrates algorithm implementation by combining input validation, API interaction, and dynamic UI updates.

In [ ]:
window.addBookToWishlist = async function () {
  const dropdown = document.getElementById('bookDropdown');
  const selectedOption = dropdown.options[dropdown.selectedIndex];
  const bookId = selectedOption.value;

  if (!bookId) {
    document.getElementById('profile-message').textContent = 'Please select a book.';
    return;
  }

  const URL = `${pythonURI}/api/wishlist/`;
  const body = { user_id: userId, book_id: parseInt(bookId) };

  try {
    const response = await fetch(URL, {
      ...fetchOptions,
      method: 'POST',
      body: JSON.stringify(body),
    });

    if (!response.ok) throw new Error(`Failed to add book: ${response.status}`);
    userWishlist = await fetchWishlist();
    displayWishlist();
  } catch (error) {
    console.error('Error adding book:', error.message);
  }
};


### Backend: Adding a Book to Wishlist
This algorithm checks if the book exists, validates that it’s not already in the wishlist, and then inserts it into the database. It demonstrates algorithm implementation by following a clear, step-by-step process to perform the desired operation.

In [ ]:
book = conn.execute('SELECT * FROM books WHERE id = ?', (book_id,)).fetchone()
if not book:
    conn.close()
    return jsonify({"error": "Book not found"}), 404

existing_entry = conn.execute(
    'SELECT * FROM wishlist WHERE user_id = ? AND book_id = ?',
    (user_id, book_id)
).fetchone()
if existing_entry:
    conn.close()
    return jsonify({"message": "Book already in wish list"}), 200

conn.execute('INSERT INTO wishlist (user_id, book_id) VALUES (?, ?)', (user_id, book_id))
conn.commit()
conn.close()
return jsonify({"message": "Book added to user's wish list"}), 201


## Iteration

### Frontend: Displaying Wishlist
The displayWishlist function iterates through the userWishlist array to dynamically update the wishlist table. This demonstrates iteration and highlights how the program outputs structured data to the user.


In [ ]:
function displayWishlist() {
  const tableBody = document.getElementById('wishlistResult');
  tableBody.innerHTML = '';
  userWishlist.forEach(book => {
    const tr = document.createElement('tr');
    const titleCell = document.createElement('td');
    const authorCell = document.createElement('td');

    titleCell.textContent = book.title;
    authorCell.textContent = book.author;

    tr.appendChild(titleCell);
    tr.appendChild(authorCell);
    tableBody.appendChild(tr);
  });
}


### Backend: Fetching Wishlist
The function uses SQL to fetch all books in the user's wishlist and converts each row into a dictionary using a list comprehension. This demonstrates iteration by processing multiple rows of data to generate the JSON response.


In [ ]:
@wishlist_api.route('/<int:user_id>', methods=['GET'])
def get_wishlist(user_id):
    conn = get_db_connection()
    books = conn.execute(
        '''
        SELECT books.id, books.title, books.author
        FROM wishlist
        JOIN books ON wishlist.book_id = books.id
        WHERE wishlist.user_id = ?
        ''',
        (user_id,)
    ).fetchall()
    conn.close()
    return jsonify([dict(book) for book in books])


## Error Handling

### Frontend: Providing Feedback
Error messages are displayed dynamically in the profile-message element. This demonstrates program purpose and functionality by handling errors and providing user feedback.

In [ ]:
document.getElementById('profile-message').textContent = `Error: ${error.message}`;

### Backend: Providing Feedback
The code provides detailed error messages when required fields are missing or a book doesn’t exist. This demonstrates program functionality by handling errors gracefully and ensuring clear communication with the frontend.

In [ ]:
if not user_id or not book_id:
    return jsonify({"error": "Missing user_id or book_id"}), 400

if not book:
    conn.close()
    return jsonify({"error": "Book not found"}), 404


## Program with a list (backend example):
The get_wishlist function uses a list of books fetched from the database and returns them as a JSON array. The list comprehension [dict(book) for book in books] processes the rows into dictionaries for easier frontend consumption.

In [ ]:
def get_wishlist(user_id):
    conn = get_db_connection()
    books = conn.execute(
        '''
        SELECT books.id, books.title, books.author
        FROM wishlist
        JOIN books ON wishlist.book_id = books.id
        WHERE wishlist.user_id = ?
        ''',
        (user_id,)
    ).fetchall()
    conn.close()
    return jsonify([dict(book) for book in books])


## Program with a Dictionary (backend example):
The add_book_to_wishlist function parses JSON input from the client into a dictionary (data), allowing the program to access specific fields like user_id and book_id. This dictionary enables dynamic, key-based data management.

In [ ]:
if request.is_json:
    data = request.get_json()
    user_id = data.get('user_id')
    book_id = data.get('book_id')